In [1]:
#Test Model
import pickle
import pandas as pd
import numpy as np
import datetime
from datetime import datetime

In [2]:
df_test = pd.read_csv('./data/test_X_clean.csv')
display(df_test.head())

,1stFlrSF,2ndFlrSF,3SsnPorch,Alley,BedroomAbvGr,BsmtCond,BsmtExposure,BsmtFinSF1,BsmtFinSF2,BsmtFinType1,...,SaleCondition_Partial,SaleType_COD,SaleType_CWD,SaleType_Con,SaleType_ConLD,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD
0,11.818680,0.000000,0.0,0.730463,1.194318,1.820334,1.540963,10.105326,7.397498,2.055642,...,0,0,0,0,0,0,0,0,0,1
1,12.943735,0.000000,0.0,0.730463,1.540963,1.820334,1.540963,11.901094,0.000000,0.000000,...,0,0,0,0,0,0,0,0,0,1
2,11.916131,11.151348,0.0,0.730463,1.540963,1.820334,1.540963,11.476685,0.000000,1.194318,...,0,0,0,0,0,0,0,0,0,1
3,11.910125,11.062536,0.0,0.730463,1.540963,1.820334,1.540963,10.749651,0.000000,1.194318,...,0,0,0,0,0,0,0,0,0,1
4,12.833625,0.000000,0.0,0.730463,1.194318,1.820334,1.540963,8.720170,0.000000,0.000000,...,0,0,0,0,0,0,0,0,0,1


In [3]:
X_test = df_test.values 
print(X_test.shape)
print(X_test[:,3])

(1459, 220)
[0.73046315 0.73046315 0.73046315 ... 0.73046315 0.73046315 0.73046315]


In [4]:
submission = pd.read_csv("./data/sample_submission.csv")
display(submission.head())
y_test_id = submission.values[:,0]
print(y_test_id.shape)
print(y_test_id)


,Id,SalePrice
0,1461,169277.052498
1,1462,187758.393989
2,1463,183583.683570
3,1464,179317.477511
4,1465,150730.079977


(1459,)
[1461. 1462. 1463. ... 2917. 2918. 2919.]


In [5]:
# load the model from disk
filename = 'stack_gen.sav'
stack_gen_model = pickle.load(open(filename, 'rb'))

filename = 'lasso.sav'
lasso_model_full_data = pickle.load(open(filename, 'rb'))

filename = 'ridge.sav'
ridge_model_full_data = pickle.load(open(filename, 'rb'))

filename = 'svr.sav'
svr_model_full_data = pickle.load(open(filename, 'rb'))

filename = 'gbr.sav'
gbr_model_full_data = pickle.load(open(filename, 'rb'))

filename = 'xgboost.sav'
xgb_model_full_data = pickle.load(open(filename, 'rb'))

filename = 'lightgbm.sav'
lgb_model_full_data = pickle.load(open(filename, 'rb'))


filename = 'rf.sav'
rf_model_full_data = pickle.load(open(filename, 'rb'))


[19:42:15] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:42:15] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:42:15] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [ ]:
class StackingAveragedModels(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, base_models, meta_model, n_folds=5):
        self.base_models = base_models
        self.meta_model = meta_model
        self.n_folds = n_folds
   
    # We again fit the data on clones of the original models
    def fit(self, X, y):
        self.base_models_ = [list() for x in self.base_models]
        self.meta_model_ = clone(self.meta_model)
        kfold = KFold(n_splits=self.n_folds, shuffle=True, random_state=156)
        
        # Train cloned base models then create out-of-fold predictions
        # that are needed to train the cloned meta-model
        out_of_fold_predictions = np.zeros((X.shape[0], len(self.base_models)))
        for i, model in enumerate(self.base_models):
            for train_index, holdout_index in kfold.split(X, y):
                instance = clone(model)
                self.base_models_[i].append(instance)
                instance.fit(X[train_index], y[train_index])
                y_pred = instance.predict(X[holdout_index])
                out_of_fold_predictions[holdout_index, i] = y_pred
                
        # Now train the cloned  meta-model using the out-of-fold predictions as new feature
        self.meta_model_.fit(out_of_fold_predictions, y)
        return self
   
    #Do the predictions of all base models on the test data and use the averaged predictions as 
    #meta-features for the final prediction which is done by the meta-model
    def predict(self, X):
        meta_features = np.column_stack([
            np.column_stack([model.predict(X) for model in base_models]).mean(axis=1)
            for base_models in self.base_models_ ])
        return self.meta_model_.predict(meta_features)

In [6]:
model_weights = {}
#model_weights = {'rf':0.01,'lasso':0.05, 'ridge':0.1, 'svr':0.1, 'gbr':0.2, 'xgb':0.15, 'lgb':0, 'stack':0.3}
model_weights = {'rf':0.01,'lasso':0.15, 'ridge':0.03, 'svr':0.01, 'gbr':0.2, 'xgb':0.15, 'lgb':0.15, 'stack':0.3}
#model_weights = {'rf':0.01,'lasso':0.15, 'ridge':0.03, 'svr':0.0, 'gbr':0.2, 'xgb':0.15, 'lgb':0.15, 'stack':0.3}

def blend_models_predict(X_pred):
    return ((model_weights['rf'] * rf_model_full_data.predict(X_pred)) + \
            (model_weights['lasso'] * lasso_model_full_data.predict(X_pred)) + \
            (model_weights['ridge'] * ridge_model_full_data.predict(X_pred)) + \
            (model_weights['svr'] * svr_model_full_data.predict(X_pred)) + \
            (model_weights['gbr'] * gbr_model_full_data.predict(X_pred)) + \
            (model_weights['xgb'] * xgb_model_full_data.predict(X_pred)) + \
            (model_weights['lgb'] * lgb_model_full_data.predict(X_pred)) + \
            (model_weights['stack'] * stack_gen_model.predict(X_pred)))


print('RMSLE score on train data:')



RMSLE score on train data:


In [10]:
#best score of .12135 with gbr model
y_test = np.floor(np.expm1(gbr_model_full_data.predict(X_test)))### Working code
print(y_test)

[122000. 153020. 192068. ... 157443. 110716. 203261.]


In [11]:
submission['SalePrice'] = y_test

In [12]:
submission.head()
submission.to_csv("submission.csv", index=False)
print('Save submission', datetime.now(),)

Save submission 2020-01-04 19:46:03.186326
